In [1]:
# !wget --load-cookies cookies.txt  https://www.kaggle.com/START-UMD/gtd/download
# !wget --load-cookies cookies.txt https://www.kaggle.com/burhanykiyakoglu/infant-mortality-fertility-income/download
# !wget --load-cookies cookies.txt https://www.kaggle.com/ibrahimmukherjee/gdp-world-bank-data/download
# !wget --load-cookies cookies.txt https://www.kaggle.com/juanumusic/countries-iso-codes/download
# !unzip download.4
# !ls
# !pip install google-cloud-bigquery==1.22.0
# !pip install  six==1.14.0




In [2]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd 
import json 



In [3]:
# 128 * 256 * 5

In [4]:

pd.set_option('display.max_columns', 500)

In [5]:
#iso country codes 
codes = pd.read_csv('wikipedia-iso-country-codes.csv').loc[:, ['English short name lower case', 'Alpha-3 code']]
codes.columns = ['Name', 'Code']


codes.head(5)

codes[codes.Name.str.contains('United')]

,Name,Code
216,"Tanzania, United Republic of",TZA
230,United Arab Emirates,ARE
231,United Kingdom,GBR
232,United States Of America,USA
233,United States Minor Outlying Islands,UMI


In [6]:
#fix country names for terror 
terror_country_map = { 'South Korea' : 'Korea, Republic of (South Korea)'  
                     , 'United States' : 'United States Of America'
                     , 'Czechoslovakia' : 'Czech Republic'
                     , 'Brunei' : 'Brunei Darussalam'
                     , 'Soviet Union' : 'Russia'
                     }


In [7]:
#terror incidents per year by country 


terror = pd.read_csv('globalterrorismdb_0718dist.csv'
#                      , nrows = 10000
                     , encoding = "ISO-8859-1"
                    , usecols = ['iyear', 'country_txt', 'success'])

terror = terror.replace({'country_txt' : terror_country_map})
terror = terror.rename(columns = {'iyear' : 'Year'})
terror = terror.merge(codes, how = 'left', left_on = 'country_txt', right_on = 'Name')

#countries excluded 
terror.loc[terror.Code.isnull(), :].country_txt.unique()

terror = terror.loc[~terror.Code.isnull(), ['Code', 'Year', 'success']]

# terror.head(5)

terror = terror.groupby(['Code', 'Year']).agg(['count', 'sum'])
terror = terror.reset_index()
terror.columns = [''.join(col) for col in terror.columns]
terror = terror.rename(columns = {'successcount' : 'incidents', 'successsum' : 'successes'})

terror.head(5)



,Code,Year,incidents,successes
0,AFG,1973,1,0
1,AFG,1979,3,3
2,AFG,1987,1,1
3,AFG,1988,11,11
4,AFG,1989,10,6


In [8]:
#population by country by year
population = pd.read_csv('PopulationPerCountry.csv' , encoding = "ISO-8859-1", skiprows = 3)

population = pd.melt(population, ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'])
# 
population = population.loc[population.value >= 0.0, ['Country Code', 'variable', 'value']]
population.columns = ['Code', 'Year', 'Population']
# population['DataType'] = 'Population'
population.Year = pd.to_numeric(population.Year)
population.head(5)





,Code,Year,Population
0,ABW,1960,54211.0
1,AFG,1960,8996351.0
2,AGO,1960,5643182.0
3,ALB,1960,1608800.0
4,AND,1960,13411.0


In [9]:
#education by country by year 

#this one from kaggle is missing data 
education = pd.read_csv('public-education-expenditure-as-share-of-gdp.csv', 
                        usecols = ['Code', 'Year', 'Public Expenditure on Education (percent of GDP)']
                       )
education.columns = ['Code', 'Year', 'EducationPctGdp']
education['DataType'] = 'EducationPctGDP'


education.head(5)


#this one is better
credentials = service_account.Credentials.from_service_account_file('gckey.json')

client = bigquery.client.Client(project = 'tpu-44747', credentials = credentials)
datasetref = client.dataset('world_bank_intl_education', project = 'bigquery-public-data')

dataset = client.get_dataset(datasetref)
    
query = """
    SELECT *
    FROM `bigquery-public-data.world_bank_intl_education.international_education`

    where indicator_name = 'Government expenditure on education as % of GDP (%)'
    and year >= 1970 
    
"""
query_job = client.query(query)  # Make an API request.

education = client.query(query).to_dataframe()
education = education.loc[:, ['country_code', 'year', 'value']]
education.columns = ['Code', 'Year', 'EducationPctGdp']

education.head(5)


,Code,Year,EducationPctGdp
0,SSD,2016,1.81094
1,BRN,2016,4.42541
2,BGD,2016,1.92634
3,KIR,1970,4.79313
4,GUY,1970,4.32898


In [10]:
    
query = """
SELECT  * FROM `bigquery-public-data.world_bank_wdi.indicators_data` 
where indicator_name = 'Intentional homicides (per 100,000 people)'
"""
query_job = client.query(query) 
homicides = client.query(query).to_dataframe().loc[:, ['country_code', 'year', 'value']]
homicides.columns = ['Code', 'Year', 'Homicidesper100k']
homicides.head(5)



,Code,Year,Homicidesper100k
0,DEU,2017,1.0
1,HTI,2010,6.8
2,PRT,2003,1.4
3,JAM,2006,48.5
4,SAU,2001,1.1


In [11]:
#mortality, fertility, income by country by year

mort = pd.read_csv('MortalityFertilityIncome.csv', 
#                    nrows = 5, 
                   encoding = "ISO-8859-1", 
                   thousands='.', 
                   decimal=',')

mort = pd.melt(mort, ['Country Name', 'Country Code', 'Region'])


mort['Data Type'] = 'InfantMortality' 
mort.loc[mort.variable.str.contains('i'), 'Data Type'] = 'IncomePerCapita'
mort.loc[mort.variable.str.contains('f'), 'Data Type'] = 'FertilityRate'
mort.variable = mort.variable.str.slice(1)
mort = mort.rename(columns = {'variable' : 'Year', 'Country Code' : 'Code'})
mort = mort.loc[:, ['Code', 'Year' ,'value', 'Data Type']]
mort.value = pd.to_numeric(mort.value)
mort.Year = pd.to_numeric(mort.Year)
mort = pd.pivot_table(mort, index = ['Code', 'Year'], columns = 'Data Type').reset_index()
mort.columns = [''.join(col).replace('value', '') for col in mort.columns]

mort.loc[mort.Code == 'USA'].head(5)


,Code,Year,FertilityRate,IncomePerCapita,InfantMortality
9012,USA,1970,2.480,4591.218634,19.9
9013,USA,1971,2.266,4891.985558,19.1
9014,USA,1972,2.010,5305.320034,18.3
9015,USA,1973,1.879,5912.942042,17.5
9016,USA,1974,1.835,6152.478813,16.7


In [51]:

#construct the main data table by cartesian of years and codes 

years =  [i for i in range(1980,2011)]
country_codes = list(codes.Code)

index = pd.MultiIndex.from_product([country_codes, years], names = ['Code', 'Year'])
data = pd.DataFrame(index = index).reset_index()

data.head(5)

#add leaf data 

data = data.merge(terror, how = 'left', on = ['Code', 'Year'])
data = data.merge(population, how = 'left', on = ['Code', 'Year'])
data = data.merge(education, how = 'left', on = ['Code', 'Year'])
data = data.merge(mort, how = 'left', on = ['Code', 'Year'])
data = data.merge(homicides, how = 'left', on = ['Code', 'Year'])

data.head(5)



,Code,Year,incidents,successes,Population,EducationPctGdp,FertilityRate,IncomePerCapita,InfantMortality,Homicidesper100k
0,AFG,1980,NaN,NaN,13248370.0,1.83639,7.449,259.338014,163.3,NaN
1,AFG,1981,NaN,NaN,13053954.0,1.90443,7.449,252.110768,158.9,NaN
2,AFG,1982,NaN,NaN,12749645.0,1.73199,7.450,NaN,154.4,NaN
3,AFG,1983,NaN,NaN,12389269.0,NaN,7.452,NaN,149.8,NaN
4,AFG,1984,NaN,NaN,12047115.0,NaN,7.455,NaN,145.6,NaN


In [52]:
#data cleanup 

#years where we have terror incident data 
#assume nans mean no incidents/sucessess at all 
print('Incident Data spans %i to %i' % (min(data.loc[~data.incidents.isnull(), 'Year'].unique()), 
      max(data.loc[~data.incidents.isnull(), 'Year'].unique() )) )
data.loc[data.incidents.isnull(), ['incidents','successes']] = 0.0 


# data.loc[data.EducationPctGdp.isnull()] 
summed = data.loc[:, ['Code', 'EducationPctGdp']].groupby(['Code']).agg('sum')
noedu = summed.loc[summed.EducationPctGdp == 0].index.values
print("countries that don't report education spending as pct of gdp" , noedu)

#keep countries that report edu data 
data = data.loc[~data.Code.isin(noedu)]

#remove countries that have a lot of nan values 
nancheck = pd.melt(data, ['Code', 'Year'])
nancheck['Nan'] = nancheck.value.isnull()
nancheck = nancheck.groupby('Code').agg({'Nan' : ['sum', 'count']})
nancheck.columns = [''.join(col).replace('Nan', '') for col in nancheck.columns]
nancheck['pct']  = nancheck['sum'] / nancheck['count']
nancheck = nancheck.loc[nancheck['pct'] <= 0.20]

print(nancheck.sort_values(by = 'pct', ascending = True))


data.loc[data['Code'].isin(nancheck.index)]



Incident Data spans 1980 to 2010
countries that don't report education spending as pct of gdp ['AIA' 'ALA' 'ANT' 'ASM' 'ATA' 'ATF' 'BIH' 'BLM' 'BVT' 'CCK' 'COK' 'CXR'
 'CYM' 'ESH' 'FLK' 'FRO' 'GGY' 'GIB' 'GLP' 'GRL' 'GUF' 'GUM' 'HMD' 'IMN'
 'IOT' 'JEY' 'MAF' 'MNE' 'MNP' 'MSR' 'MTQ' 'MYT' 'NCL' 'NFK' 'NGA' 'NIU'
 'NRU' 'PCN' 'PNG' 'PRI' 'PRK' 'PYF' 'REU' 'SGS' 'SHN' 'SJM' 'SOM' 'SPM'
 'SUR' 'TCA' 'TKL' 'TKM' 'TWN' 'UMI' 'UZB' 'VAT' 'VIR' 'WLF']
       sum  count       pct
Code                       
NOR   10.0    248  0.040323
IRL   11.0    248  0.044355
FRA   11.0    248  0.044355
FIN   12.0    248  0.048387
AUT   12.0    248  0.048387
GBR   12.0    248  0.048387
CHE   12.0    248  0.048387
PRT   12.0    248  0.048387
ISR   13.0    248  0.052419
THA   13.0    248  0.052419
NLD   13.0    248  0.052419
KOR   14.0    248  0.056452
CYP   14.0    248  0.056452
CAN   15.0    248  0.060484
MYS   17.0    248  0.068548
NZL   17.0    248  0.068548
DNK   18.0    248  0.072581
ESP   18.0    248  0

,Code,Year,incidents,successes,Population,EducationPctGdp,FertilityRate,IncomePerCapita,InfantMortality,Homicidesper100k
62,ALB,1980,0.0,0.0,2671997.0,NaN,3.621,NaN,63.4,NaN
63,ALB,1981,1.0,1.0,2726056.0,NaN,3.530,NaN,59.1,NaN
64,ALB,1982,0.0,0.0,2784278.0,NaN,3.452,NaN,55.4,NaN
65,ALB,1983,0.0,0.0,2843960.0,NaN,3.383,NaN,51.7,NaN
66,ALB,1984,0.0,0.0,2904429.0,NaN,3.323,519.823819,48.4,NaN
67,ALB,1985,0.0,0.0,2964762.0,NaN,3.269,522.360545,45.3,NaN
68,ALB,1986,0.0,0.0,3022635.0,NaN,3.217,624.374607,42.5,NaN
69,ALB,1987,0.0,0.0,3083605.0,NaN,3.164,595.000780,40.1,NaN
70,ALB,1988,0.0,0.0,3142336.0,NaN,3.108,575.732744,38.0,NaN
71,ALB,1989,0.0,0.0,3227943.0,NaN,3.046,611.552843,36.2,NaN


In [14]:
data.to_csv('midtermdata.csv')

# Datasets Included in Analysis 

Datasets were chosen to cohesively answer the four main questions listed below for a specific country and time period, with granularity in years. This section will explain how each dataset contributes to 1 or more questions. 
1. Is infant mortality improving, stable, or getting worse? 
2. Is income rising, stagnant, or falling?
3. Does one country or another seem more of less stable than ohers and why do you say this? 
4. What changes do you predict for these countries and why? 

bigquery-public-data.world_bank_intl_education.international_education
This dataset was chosen to help answer questions 3 and 4. Education spending may suggest stability and/or affect future outcomes. 

https://www.kaggle.com/juanumusic/countries-iso-codes/download
This dataset was chosen to help all questions by providing a standardized list of country codes to align all datasets. 

https://www.kaggle.com/burhanykiyakoglu/infant-mortality-fertility-income/download
This dataset directly answers 1 and 2. It can serve as a baseline for question 4.  

https://www.kaggle.com/ibrahimmukherjee/gdp-world-bank-data/download
This population dataset dataset helps adjust other datasets by population. Namely, it is used to adjust the terrorism dataset to show per capita incidents. This helps for question 3. 

https://www.kaggle.com/START-UMD/gtd/download
This terrorism dataset helps answer question 3. We can assume that fewer terrorist attacks are indicative of a more stable country. It may also impact question 4 as a predictor of future outcomes. 

bigquery-public-data.world_bank_wdi.indicators_data
This dataset was chose to get crime related data, specifically homicdes per 100,000 people. This will help answer question 3, where more homicides can be considered to be less stable.


# Steps to Prepare Datasets 
In general, all datasets were processed into a dataframe with the fields Country Code, Year, Metric 1, Metric 2, Metric N. This output will be easier to link together.  

bigquery-public-data.world_bank_intl_education.international_education
The data needed to be queried using the bigquery api. Column names needed to be changed. No additional transformations needed as data was already in proper tabular format. Data was converted to a pandas dataframe for compatability. 

https://www.kaggle.com/burhanykiyakoglu/infant-mortality-fertility-income/download
The data needed to be read into a pandas dataframe. The years attribute was stored in the columns, rather than the rows. This was fixed by using pd.melt, which turned the years columns into a single column, with 1 row per each year. The 3 metrics were set up as 3 columns. 

https://www.kaggle.com/ibrahimmukherjee/gdp-world-bank-data/download
this dataset was already in the desired format. After reading the csv into a pandas dataframe, columns were renamed. 

https://www.kaggle.com/START-UMD/gtd/download
This dataset required mapping of country names to country codes. Mapping was done using the ISO country codes with specific mappings for certain countries, such as 'Czechoslovakia' to 'Czech Republic'. Because each row represents a single incident, only incidents with a non null value for sucess (0 or 1) were included. The rows were aggregated by country and year to create a dataset that counted for each country and year the number of incidents and successful incidents. 

bigquery-public-data.world_bank_wdi.indicators_data
The data needed to be queried using the bigquery api. Column names needed to be changed. No additional transformations needed as data was already in proper tabular format. Data was converted to a pandas dataframe for compatability. 

Finally, for all datasets, we want to keep only countries that have a low percentage of nan values. After datasets are linked, only countries with at 20% nan values or less are kept. This gives us approximately 120 countries to analyze. We also restrict the time series to years between 1980 and 2010. 

# Steps to Link Datasets 
To simplify analysis, all datasets will use the primary keys of Country Code and Year. This means that no country and year combination can appear twice for the same metric. In order to ensure no gaps in the final dataset, a cartesian product of country codes and years (1970 to 2020) was used as the base dataset. All other datasets are joined to the base dataset via a left join on the country codes and years. The final output is a tabular dataset that does not require any additional joins. The main benefit is that gaps in the data are easy to identify, for some countries will have Nan values for certain years and actual values for other years. 

Additional metrics can be added from additional datasets, so long as the dataset is processed into the format Country Code, Year, and Metric. 
